In [1]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [2]:
from helper import *
from datasetcreator import *
from dependencies import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata

User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [3]:
def give_text_feats_cv_elite(review_data):
    review_data_ne = review_data.loc[review_data['iselite'] == True]
    bid_2_stars = review_data_ne.groupby('business_id')['stars'].apply(list).to_dict()
    bid_2_textrev = review_data_ne.groupby('business_id')['text'].apply(list).to_dict()
    bid_2_text = {i:' '.join(j) for i,j in bid_2_textrev.items() if all(isinstance(x, str) for x in j)}
    cv = CountVectorizer(min_df = 400, stop_words="english")
    textfeats = cv.fit_transform(bid_2_text.values())
    print("Size of Vocabulary: ", len(cv.vocabulary_))
    bid_2_textenc = {a:b.toarray()[0] for a,b in zip(bid_2_text.keys(),textfeats)}
    return cv, textfeats, bid_2_stars, review_data_ne, bid_2_textenc

In [4]:
cv, textfeats, bid_2_stars_ne, review_data_ne, bid_2_textenc = give_text_feats_cv_elite(review_data)
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()

Size of Vocabulary:  9763


In [5]:
import seaborn as sns
sns.set_style('whitegrid')
def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()

In [6]:
%matplotlib qt
plot_10_most_common_words(textfeats,cv)


In [7]:
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below (use int values below 15)
number_topics = 10
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics)
lda.fit(textfeats)

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, cv, number_words)

Topics found via LDA:

Topic #0:
store like just place great location don good selection time

Topic #1:
coffee tea place like good drink just really drinks milk

Topic #2:
room hotel vegas strip like nice just stay casino time

Topic #3:
place like great time bar just really good people fun

Topic #4:
breakfast good cream place like just ice chocolate really time

Topic #5:
food good place like chicken rice just sushi really service

Topic #6:
food good place great like just chicken really cheese fries

Topic #7:
food good great place just like service restaurant really menu

Topic #8:
et le la est les pour montreal une des en

Topic #9:
time just service car did like great place ve got


In [8]:
print_topics(lda, cv, 10)


Topic #0:
store like just place great location don good selection time

Topic #1:
coffee tea place like good drink just really drinks milk

Topic #2:
room hotel vegas strip like nice just stay casino time

Topic #3:
place like great time bar just really good people fun

Topic #4:
breakfast good cream place like just ice chocolate really time

Topic #5:
food good place like chicken rice just sushi really service

Topic #6:
food good place great like just chicken really cheese fries

Topic #7:
food good great place just like service restaurant really menu

Topic #8:
et le la est les pour montreal une des en

Topic #9:
time just service car did like great place ve got


In [1]:
lda

NameError: name 'lda' is not defined